In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras.layers import Dense, LSTM, Input, TimeDistributed, Bidirectional, Activation
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model

In [2]:
# bd sn ht mt lt oh ch cr ri
checking_arr = [
    [1, 0, 0, 0, 0, 0, 0, 0, 0], #0
    [0, 1, 0, 0, 0, 0, 0, 0, 0], #1
    # [0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0], #2
    [0, 0, 0, 0, 1, 0, 0, 0, 0], #3
    [0, 0, 0, 0, 0, 1, 0, 0, 0], #4
    [0, 0, 0, 0, 0, 0, 1, 0, 0], #5
    # [0, 0, 0, 0, 0, 0, 0, 1, 0], 
    [0, 0, 0, 0, 0, 0, 0, 0, 1], #6
    [1, 0, 0, 0, 0, 0, 1, 0, 0], #7
    [1, 0, 0, 0, 0, 1, 0, 0, 0], #8
    [1, 0, 0, 0, 0, 0, 0, 0, 1], #9
    [1, 0, 0, 0, 1, 0, 0, 0, 0], #10
    [1, 0, 0, 0, 0, 0, 0, 1, 0], #11
    [1, 1, 0, 0, 0, 0, 0, 0, 0], #12
    [0, 1, 0, 0, 0, 0, 1, 0, 0], #13
    [0, 1, 0, 0, 0, 1, 0, 0, 0], #14
    [0, 1, 0, 0, 0, 0, 0, 0, 1], #15
    [0, 1, 0, 0, 1, 0, 0, 0, 0], #16
    [0, 1, 0, 0, 0, 0, 0, 1, 0], #17
    [0, 0, 0, 1, 1, 0, 0, 0, 0], #18
    [1, 1, 0, 0, 0, 0, 1, 0, 0], #19
    [1, 1, 0, 0, 0, 1, 0, 0, 0], #20
    [1, 1, 0, 0, 0, 0, 0, 0, 1], #21
    [0, 0, 0, 0, 0, 0, 0, 0, 0] #22
]


seven_kind_arr = [
   # bd sn mt lt oh ch ri 
    [1, 0, 0, 0, 0, 0, 0], #0
    [0, 1, 0, 0, 0, 0, 0], #1
    [0, 0, 1, 0, 0, 0, 0], #2
    [0, 0, 0, 1, 0, 0, 0], #3
    [0, 0, 0, 0, 1, 0, 0], #4
    [0, 0, 0, 0, 0, 1, 0], #5
    [0, 0, 0, 0, 0, 0, 1], #6
    [1, 0, 0, 0, 0, 1, 0], #7
    [1, 0, 0, 0, 1, 0, 0], #8
    [1, 0, 0, 0, 0, 0, 1], #9
    [1, 0, 0, 1, 0, 0, 0], #10
    [1, 1, 0, 0, 0, 0, 0], #11
    [0, 1, 0, 0, 0, 1, 0], #12
    [0, 1, 0, 0, 1, 0, 0], #13
    [0, 1, 0, 0, 0, 0, 1], #14
    [0, 1, 0, 1, 0, 0, 0], #15
    [0, 1, 0, 0, 0, 0, 1], #16
    [0, 0, 1, 1, 0, 0, 0], #17
    [1, 1, 0, 0, 0, 1, 0], #18
    [1, 1, 0, 0, 1, 0, 0], #19
    [1, 1, 0, 0, 0, 0, 1], #20
    [0, 0, 0, 0, 0, 0, 0]
]




In [3]:
def return_pattern(i):
    return seven_kind_arr[i.tolist().index(1)]

In [19]:
train_x = []
train_y = []

for i in range(1,6116):
    tmp = pd.read_csv('./single_label_dataset/' + str(i) + '.csv.gzip', compression='gzip').to_numpy()
    # imsi = []
    # for j in range(31):
    #     imsi.append(return_pattern(tmp[j]))
    # train_x.append(imsi)
    # train_y.append(return_pattern(tmp[-1]))
    train_x.append(tmp[:31])
    train_y.append(tmp[32])
    train_x.append(tmp[-32:-1])
    train_y.append(tmp[-1])

train_x = np.array(train_x)
train_y = np.array(train_y)



In [ ]:
y1 = np.swapaxes(train_y,0,1)[0]
y2 = np.swapaxes(train_y,0,1)[1]
y3 = np.swapaxes(train_y,0,1)[2]
y4 = np.swapaxes(train_y,0,1)[3]
y5 = np.swapaxes(train_y,0,1)[4]
y6 = np.swapaxes(train_y,0,1)[5]
y7 = np.swapaxes(train_y,0,1)[6]
# y8 = np.swapaxes(train_y,0,1)[7]
# y9 = np.swapaxes(train_y,0,1)[8]

In [ ]:
def create_model(drp):
    inputt = Input(shape=(63,7))
    
    lstm1 = Bidirectional(LSTM(512, return_sequences=True, dropout=drp , recurrent_dropout=drp))(inputt)
    lstm2 = Bidirectional(LSTM(128, return_sequences=True, dropout=drp , recurrent_dropout=drp))(lstm1)
    lstm3 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)

    out1 = Dense(1, activation='sigmoid', name='bd')(lstm3)
    out2 = Dense(1, activation='sigmoid', name='sn')(lstm3)
    out3 = Dense(1, activation='sigmoid', name='mt')(lstm3)
    out4 = Dense(1, activation='sigmoid', name='lt')(lstm3)
    out5 = Dense(1, activation='sigmoid', name='oh')(lstm3)
    out6 = Dense(1, activation='sigmoid', name='ch')(lstm3)
    out7 = Dense(1, activation='sigmoid', name='ri')(lstm3)
    # out8 = Dense(1, activation='sigmoid', name='cr')(lstm3)
    # out9 = Dense(1, activation='sigmoid', name='ri')(lstm3)

    # return Model(inputs=inputt, outputs=[out1,out2,out3,out4,out5,out6,out7,out8,out9])
    return Model(inputs=inputt, outputs=[out1,out2,out3,out4,out5,out6,out7])

In [9]:
# model = create_model(0.1)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)

In [ ]:
model.summary()

In [ ]:
model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.RMSprop(), metrics=['accuracy'])

In [ ]:
with tf.device("/gpu:0"):
    model.fit(x=train_x,y=[y1,y2,y3,y4,y5,y6,y7], epochs=4000, callbacks=callback, validation_split=0.1)

In [ ]:
model.save('./chooseok/multi_output_1_drp0.1.h5')

In [ ]:
model = create_model(0.2)
model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.RMSprop(), metrics=['accuracy'])

In [ ]:
with tf.device("/gpu:0"):
    model.fit(x=train_x,y=[y1,y2,y3,y4,y5,y6,y7], epochs=4000, callbacks=callback, validation_split=0.1)

In [ ]:
model.save('./chooseok/multi_output_1_drp0.2.h5')

In [ ]:
model = create_model(0.4)
model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.RMSprop(), metrics=['accuracy'])

In [ ]:
with tf.device("/gpu:0"):
    model.fit(x=train_x,y=[y1,y2,y3,y4,y5,y6,y7], epochs=4000, callbacks=callback, validation_split=0.1)

In [ ]:
model.save('./chooseok/multi_output_1_drp0.4.h5')

In [ ]:
def create_other_model(drp):
    inputt = Input(shape=(63,7))
    
    lstm1 = Bidirectional(LSTM(512, return_sequences=True, dropout=drp , recurrent_dropout=drp))(inputt)
    lstm2 = Bidirectional(LSTM(128, return_sequences=True, dropout=drp , recurrent_dropout=drp))(lstm1)

    lstm_1 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out1 = Dense(1, activation='sigmoid', name='bd')(lstm_1)
    
    lstm_2 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out2 = Dense(1, activation='sigmoid', name='sn')(lstm_2)

    lstm_3 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out3 = Dense(1, activation='sigmoid', name='mt')(lstm_3)

    lstm_4 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out4 = Dense(1, activation='sigmoid', name='lt')(lstm_4)

    lstm_5 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out5 = Dense(1, activation='sigmoid', name='oh')(lstm_5)

    lstm_6 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out6 = Dense(1, activation='sigmoid', name='ch')(lstm_6)

    lstm_7 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out7 = Dense(1, activation='sigmoid', name='ri')(lstm_7)
    # out8 = Dense(1, activation='sigmoid', name='cr')(lstm3)
    # out9 = Dense(1, activation='sigmoid', name='ri')(lstm3)

    # return Model(inputs=inputt, outputs=[out1,out2,out3,out4,out5,out6,out7,out8,out9])
    return Model(inputs=inputt, outputs=[out1,out2,out3,out4,out5,out6,out7])

In [ ]:
model = create_model(0.2)
model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.RMSprop(), metrics=['accuracy'])

In [ ]:
model = create_other_model(0.2)
model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.RMSprop(), metrics=['accuracy'])

In [ ]:
with tf.device("/gpu:0"):
    model.fit(x=train_x,y=[y1,y2,y3,y4,y5,y6,y7], epochs=4000, callbacks=callback, validation_split=0.1)

In [ ]:
model.save('./chooseok/multi_output_2_drp0.2.h5')

In [ ]:
model = create_other_model(0.4)
model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.RMSprop(), metrics=['accuracy'])

In [ ]:
with tf.device("/gpu:0"):
    model.fit(x=train_x,y=[y1,y2,y3,y4,y5,y6,y7], epochs=4000, callbacks=callback, validation_split=0.1)

In [ ]:
model.save('./chooseok/multi_output_2_drp0.4.h5')

In [ ]:
def create_l2_model(drp):
    inputt = Input(shape=(63,7))
    
    lstm1 = Bidirectional(LSTM(512, return_sequences=True, dropout=drp , recurrent_dropout=drp, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01)))(inputt)
    lstm2 = Bidirectional(LSTM(128, return_sequences=True, dropout=drp , recurrent_dropout=drp, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01)))(lstm1)

    lstm_1 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out1 = Dense(1, activation='sigmoid', name='bd')(lstm_1)
    
    lstm_2 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out2 = Dense(1, activation='sigmoid', name='sn')(lstm_2)

    lstm_3 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out3 = Dense(1, activation='sigmoid', name='mt')(lstm_3)

    lstm_4 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out4 = Dense(1, activation='sigmoid', name='lt')(lstm_4)

    lstm_5 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out5 = Dense(1, activation='sigmoid', name='oh')(lstm_5)

    lstm_6 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out6 = Dense(1, activation='sigmoid', name='ch')(lstm_6)

    lstm_7 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out7 = Dense(1, activation='sigmoid', name='ri')(lstm_7)
    # out8 = Dense(1, activation='sigmoid', name='cr')(lstm3)
    # out9 = Dense(1, activation='sigmoid', name='ri')(lstm3)

    # return Model(inputs=inputt, outputs=[out1,out2,out3,out4,out5,out6,out7,out8,out9])
    return Model(inputs=inputt, outputs=[out1,out2,out3,out4,out5,out6,out7])

In [ ]:
model = create_l2_model(0.2)
model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [ ]:
with tf.device("/gpu:0"):
    model.fit(x=train_x,y=[y1,y2,y3,y4,y5,y6,y7], epochs=4000, callbacks=callback, validation_split=0.1)

In [ ]:
model.save('./chooseok/multi_output_l2_drp0.2.h5')

In [23]:
units = 128
dropout = 0.25

model = tf.keras.models.Sequential()
forward_layer = LSTM(units=units, dropout=dropout, recurrent_dropout=dropout)
backward_layer = LSTM(units=units, activation='relu', go_backwards=True, dropout=dropout, recurrent_dropout=dropout)
model.add(Bidirectional(forward_layer, backward_layer=backward_layer, input_shape=(31,22)))
model.add(Dense(22, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
train_x = []
train_y = []

for i in range(1,6000):
    tmp = pd.read_csv('./single_label_dataset/' + str(i) + '.csv.gzip', compression='gzip').to_numpy()

    train_x.append(tmp[:63])
    train_y.append(tmp[-1])

train_x = np.array(train_x)
train_y = np.array(train_y)



In [24]:
with tf.device('/gpu:0'):
    model.fit(x=train_x, y=train_y, epochs=5000, callbacks=callback, validation_split=0.1)

Epoch 1/5000
344/344 [==============================] - 117s 334ms/step - loss: 1.9662 - accuracy: 0.3729 - val_loss: 1.9113 - val_accuracy: 0.3892
Epoch 2/5000
344/344 [==============================] - 117s 341ms/step - loss: 1.7975 - accuracy: 0.3936 - val_loss: 1.8163 - val_accuracy: 0.4031
Epoch 3/5000
344/344 [==============================] - 120s 349ms/step - loss: 1.7325 - accuracy: 0.4047 - val_loss: 1.7665 - val_accuracy: 0.4113
Epoch 4/5000
344/344 [==============================] - 118s 342ms/step - loss: 1.6976 - accuracy: 0.4188 - val_loss: 1.7105 - val_accuracy: 0.4350
Epoch 5/5000
344/344 [==============================] - 116s 336ms/step - loss: 1.6620 - accuracy: 0.4246 - val_loss: 1.6793 - val_accuracy: 0.4358
Epoch 6/5000
344/344 [==============================] - 114s 333ms/step - loss: 1.6330 - accuracy: 0.4295 - val_loss: 1.6573 - val_accuracy: 0.4383
Epoch 7/5000
344/344 [==============================] - 113s 329ms/step - loss: 1.6113 - accuracy: 0.4289 - val_

In [17]:
model.save('./chooseok/32(2).h5')

In [19]:
units = 128
dropout = 0.2

model = tf.keras.models.Sequential()
forward_layer = tf.keras.layers.GRU(units=units, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))
backward_layer = tf.keras.layers.GRU(units=units, activation='relu', go_backwards=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))
model.add(Bidirectional(forward_layer, backward_layer=backward_layer, input_shape=(63,7)))
model.add(Dense(7, activation='sigmoid'))

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [20]:
with tf.device('/gpu:0'):
    model.fit(x=train_x, y=train_y, epochs=5000, callbacks=callback, validation_split=0.1)

Epoch 1/5000
169/169 [==============================] - 108s 628ms/step - loss: 0.9282 - accuracy: 0.2313 - val_loss: 0.3486 - val_accuracy: 0.3000
Epoch 2/5000
169/169 [==============================] - 104s 613ms/step - loss: 0.3267 - accuracy: 0.2758 - val_loss: 0.3240 - val_accuracy: 0.2483
Epoch 3/5000
169/169 [==============================] - 104s 618ms/step - loss: 0.3185 - accuracy: 0.2821 - val_loss: 0.3201 - val_accuracy: 0.3067
Epoch 4/5000
169/169 [==============================] - 106s 628ms/step - loss: 0.3157 - accuracy: 0.2925 - val_loss: 0.3172 - val_accuracy: 0.3033
Epoch 5/5000
169/169 [==============================] - 104s 616ms/step - loss: 0.3131 - accuracy: 0.3002 - val_loss: 0.3179 - val_accuracy: 0.2900
Epoch 6/5000
169/169 [==============================] - 103s 607ms/step - loss: 0.3106 - accuracy: 0.2984 - val_loss: 0.3124 - val_accuracy: 0.2983
Epoch 7/5000
169/169 [==============================] - 102s 603ms/step - loss: 0.3074 - accuracy: 0.3021 - val_

In [21]:
model.save('./chooseok/GRU_bin.h5')

In [22]:
units = 128
dropout = 0.2

model = tf.keras.models.Sequential()
forward_layer = tf.keras.layers.GRU(units=units, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))
backward_layer = tf.keras.layers.GRU(units=units, activation='relu', go_backwards=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))
model.add(Bidirectional(forward_layer, backward_layer=backward_layer, input_shape=(63,7)))
model.add(Dense(7, activation='sigmoid'))

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [23]:
with tf.device('/gpu:0'):
    model.fit(x=train_x, y=train_y, epochs=500, validation_split=0.1)

Epoch 1/500
169/169 [==============================] - 103s 597ms/step - loss: 0.9282 - accuracy: 0.2426 - val_loss: 0.3501 - val_accuracy: 0.2917
Epoch 2/500
169/169 [==============================] - 102s 601ms/step - loss: 0.3264 - accuracy: 0.2773 - val_loss: 0.3231 - val_accuracy: 0.2800
Epoch 3/500
169/169 [==============================] - 102s 605ms/step - loss: 0.3183 - accuracy: 0.2862 - val_loss: 0.3224 - val_accuracy: 0.2633
Epoch 4/500
169/169 [==============================] - 102s 603ms/step - loss: 0.3145 - accuracy: 0.2991 - val_loss: 0.3166 - val_accuracy: 0.3017
Epoch 5/500
169/169 [==============================] - 105s 622ms/step - loss: 0.3122 - accuracy: 0.2988 - val_loss: 0.3137 - val_accuracy: 0.3017
Epoch 6/500
169/169 [==============================] - 109s 641ms/step - loss: 0.3098 - accuracy: 0.3045 - val_loss: 0.3110 - val_accuracy: 0.3017
Epoch 7/500
169/169 [==============================] - 102s 604ms/step - loss: 0.3072 - accuracy: 0.3010 - val_loss: 0

In [24]:
model.save('./chooseok/GRU_500.h5')

In [ ]:
train_x = []
train_y = []

for i in range(1,6000):
    tmp = pd.read_csv('./single_label_dataset/' + str(i) + '.csv.gzip', compression='gzip').to_numpy()
    train_x.append(tmp[:63])
    train_y.append(tmp[-1])

train_x = np.array(train_x)
train_y = np.array(train_y)



In [ ]:
units = 128
dropout = 0.2

model = tf.keras.models.Sequential()
forward_layer = tf.keras.layers.GRU(units=units, dropout=dropout, recurrent_dropout=dropout)
backward_layer = tf.keras.layers.GRU(units=units, activation='relu', go_backwards=True, dropout=dropout, recurrent_dropout=dropout)
model.add(Bidirectional(forward_layer, backward_layer=backward_layer, input_shape=(63,22)))
model.add(Dense(22, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
with tf.device('/gpu:0'):
    model.fit(x=train_x, y=train_y, epochs=5000, callbacks=callback, validation_split=0.1)

In [ ]:
model.save('./chooseok/GRU_Cat_Single.h5')

------


In [ ]:
model.save('./chooseok/bidunits128dropout0.2l2norm.h5')

In [ ]:
units = 64
dropout = 0.4

encoder_inputs = Input(shape=(63,22))
encoder_lstm = LSTM(units=units, return_state=True, dropout=dropout, recurrent_dropout=dropout)

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

encoder_states = [state_h, state_c]


decoder_inputs = Input(shape=(None,22))
decoder_lstm = LSTM(units=units, return_state=True, dropout=dropout, recurrent_dropout=dropout)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

decoder_dense = Dense(22, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)


In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
with tf.device('/gpu:0'):
    model.fit(x=[train_x, dec], y=train_y, epochs=5000, callbacks=callback, validation_split=0.1)

In [ ]:
model.save('./chooseok/unit64dropout0.4.h5')

In [ ]:
units = 512
dropout = 0.4

encoder_inputs = Input(shape=(63,22))
encoder_lstm = LSTM(units=units, return_state=True, dropout=dropout, recurrent_dropout=dropout)

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

encoder_states = [state_h, state_c]


decoder_inputs = Input(shape=(None,22))
decoder_lstm = LSTM(units=units, return_state=True, dropout=dropout, recurrent_dropout=dropout)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

decoder_dense = Dense(22, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)


In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
with tf.device('/gpu:0'):
    model.fit(x=[train_x, dec], y=train_y, epochs=5000, callbacks=callback, validation_split=0.1)

In [ ]:
model.save('./chooseok/unit512dropout0.4.h5')